---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

In [67]:
import pandas as pd
import numpy as np

def blight_model():
    
    # Your code here
    
    return # Your answer here

In [131]:
df = pd.read_csv('train.csv', encoding = 'ISO-8859-1')
addresses_df = pd.read_csv('addresses.csv')
latlons = pd.read_csv('latlons.csv')

C:\Users\Admin\Anaconda3\envs\TensorFlow-GPU-Keras\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [132]:
Y = df.compliance

In [133]:
X = df.drop(columns=['compliance'])

In [134]:
X_train.columns

Index(['ticket_id', 'agency_name', 'inspector_name', 'violator_name',
       'violation_street_number', 'violation_street_name',
       'violation_zip_code', 'mailing_address_str_number',
       'mailing_address_str_name', 'city', 'state', 'zip_code',
       'non_us_str_code', 'country', 'ticket_issued_date', 'hearing_date',
       'violation_code', 'violation_description', 'disposition', 'fine_amount',
       'admin_fee', 'state_fee', 'late_fee', 'discount_amount',
       'clean_up_cost', 'judgment_amount', 'grafitti_status'],
      dtype='object')

In [119]:
print(addresses_df.shape)
addresses_df.head()

(311307, 2)


,ticket_id,address
0,22056,"2900 tyler, Detroit MI"
1,27586,"4311 central, Detroit MI"
2,22062,"1449 longfellow, Detroit MI"
3,22084,"1441 longfellow, Detroit MI"
4,22093,"2449 churchill, Detroit MI"


In [121]:
print(latlons.shape)
latlons.head()

(121769, 3)


,address,lat,lon
0,"4300 rosa parks blvd, Detroit MI 48208",42.346169,-83.079962
1,"14512 sussex, Detroit MI",42.394657,-83.194265
2,"3456 garland, Detroit MI",42.373779,-82.986228
3,"5787 wayburn, Detroit MI",42.403342,-82.957805
4,"5766 haverhill, Detroit MI",42.407255,-82.946295


In [ ]:
#addresses_df.merge(latlons, )

In [135]:
# remofe columns to prevent data leakage
X.drop(columns=['payment_amount', 'payment_date', 'payment_status', 'balance_due', 'collection_status', 'compliance_detail'], inplace=True)

In [136]:
print(X.shape)
X

(250306, 27)


,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,violation_description,disposition,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,grafitti_status
0,22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900.0,TYLER,NaN,3.0,S. WICKER,CHICAGO,...,Failure of owner to obtain certificate of comp...,Responsible by Default,250.0,20.0,10.0,25.0,0.0,0.0,305.0,NaN
1,27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",4311.0,CENTRAL,NaN,2959.0,Martin Luther King,Detroit,...,Failed To Secure Permit For Lawful Use Of Buil...,Responsible by Determination,750.0,20.0,10.0,75.0,0.0,0.0,855.0,NaN
2,22062,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","SANDERS, DERRON",1449.0,LONGFELLOW,NaN,23658.0,P.O. BOX,DETROIT,...,Failure of owner to obtain certificate of comp...,Not responsible by Dismissal,250.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,22084,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","MOROSI, MIKE",1441.0,LONGFELLOW,NaN,5.0,ST. CLAIR,DETROIT,...,Failure of owner to obtain certificate of comp...,Not responsible by City Dismissal,250.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,22093,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","NATHANIEL, NEAL",2449.0,CHURCHILL,NaN,7449.0,CHURCHILL,DETROIT,...,Failure of owner to obtain certificate of comp...,Not responsible by Dismissal,250.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
5,22046,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","KASIMU, UKWELI",6478.0,NORTHFIELD,NaN,2755.0,E. 17TH,LOG BEACH,...,Failure of owner to obtain certificate of comp...,Responsible by Default,250.0,20.0,10.0,25.0,0.0,0.0,305.0,NaN
6,18738,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Deerwood Development Group Inc, Deer",8027.0,BRENTWOOD,NaN,476.0,Garfield,Clinton,...,Failed To Secure Permit For Lawful Use Of Land,Responsible by Default,750.0,20.0,10.0,75.0,0.0,0.0,855.0,NaN
7,18735,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Rafee Auto Services L.L.C., RAF",8228.0,MT ELLIOTT,NaN,8228.0,Mt. Elliott,Detroit,...,Noncompliance/Grant Condition/BZA/BSE,Responsible by Default,100.0,20.0,10.0,10.0,0.0,0.0,140.0,NaN
8,18733,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Rafee Auto Services L.L.C., RAF",8228.0,MT ELLIOTT,NaN,8228.0,Mt. Elliott,Detroit,...,Noncompliance/Grant Condition/BZA/BSE,Responsible by Default,100.0,20.0,10.0,10.0,0.0,0.0,140.0,NaN
9,28204,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Inc, Nanno",15307.0,SEVEN MILE,NaN,1537.0,E. Seven Mile,Detroit,...,Failed To Secure Permit For Lawful Use Of Buil...,Responsible by Default,750.0,20.0,10.0,75.0,0.0,0.0,855.0,NaN


In [137]:
# number of unique values
cols_to_encode = []
for col in X.columns:
    print(col, 'coltype - ', X[col].dtype, ' - ', len(X[col].unique()))
    if (X[col].dtype == 'object'):
        cols_to_encode.append(col)

ticket_id coltype -  int64  -  250306
agency_name coltype -  object  -  5
inspector_name coltype -  object  -  173
violator_name coltype -  object  -  119993
violation_street_number coltype -  float64  -  19175
violation_street_name coltype -  object  -  1791
violation_zip_code coltype -  float64  -  1
mailing_address_str_number coltype -  float64  -  15827
mailing_address_str_name coltype -  object  -  37897
city coltype -  object  -  5184
state coltype -  object  -  60
zip_code coltype -  object  -  5643
non_us_str_code coltype -  object  -  3
country coltype -  object  -  5
ticket_issued_date coltype -  object  -  86979
hearing_date coltype -  object  -  6223
violation_code coltype -  object  -  235
violation_description coltype -  object  -  258
disposition coltype -  object  -  9
fine_amount coltype -  float64  -  44
admin_fee coltype -  float64  -  2
state_fee coltype -  float64  -  2
late_fee coltype -  float64  -  37
discount_amount coltype -  float64  -  13
clean_up_cost colty

In [138]:
cols_to_encode

['agency_name',
 'inspector_name',
 'violator_name',
 'violation_street_name',
 'mailing_address_str_name',
 'city',
 'state',
 'zip_code',
 'non_us_str_code',
 'country',
 'ticket_issued_date',
 'hearing_date',
 'violation_code',
 'violation_description',
 'disposition',
 'grafitti_status']

In [142]:
X_encoded = X.copy()

for col in cols_to_encode:
    X_encoded[col] = X[col].astype('category').cat.codes

In [143]:
X_encoded.head()

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,violation_description,disposition,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,grafitti_status
0,22056,0,137,52230,2900.0,1640,NaN,3.0,27078,682,...,77,6,250.0,20.0,10.0,25.0,0.0,0.0,305.0,-1
1,27586,0,166,77135,4311.0,306,NaN,2959.0,18442,1204,...,54,7,750.0,20.0,10.0,75.0,0.0,0.0,855.0,-1
2,22062,0,137,94185,1449.0,1021,NaN,23658.0,21873,1104,...,77,2,250.0,0.0,0.0,0.0,0.0,0.0,0.0,-1
3,22084,0,137,74625,1441.0,1021,NaN,5.0,28889,1104,...,77,0,250.0,0.0,0.0,0.0,0.0,0.0,0.0,-1
4,22093,0,137,77944,2449.0,343,NaN,7449.0,5401,1104,...,77,2,250.0,0.0,0.0,0.0,0.0,0.0,0.0,-1


In [144]:
print(X_encoded.isna().values.any())
print(X_encoded.isnull().values.any())
print(X_encoded.shape)

True
True
(250306, 27)


In [145]:
# missing values before handling
X_encoded.isnull().sum()

ticket_id                          0
agency_name                        0
inspector_name                     0
violator_name                      0
violation_street_number            0
violation_street_name              0
violation_zip_code            250306
mailing_address_str_number      3602
mailing_address_str_name           0
city                               0
state                              0
zip_code                           0
non_us_str_code                    0
country                            0
ticket_issued_date                 0
hearing_date                       0
violation_code                     0
violation_description              0
disposition                        0
fine_amount                        1
admin_fee                          0
state_fee                          0
late_fee                           0
discount_amount                    0
clean_up_cost                      0
judgment_amount                    0
grafitti_status                    0
d

In [146]:
X_encoded.mailing_address_str_number.fillna(value=0, inplace=True)

X_encoded.drop(columns=['violation_zip_code'], inplace=True)

X_encoded.fine_amount.fillna(value=0, inplace = True)

In [150]:
# missing values after handling
X_encoded.isnull().sum()

ticket_id                     0
agency_name                   0
inspector_name                0
violator_name                 0
violation_street_number       0
violation_street_name         0
mailing_address_str_number    0
mailing_address_str_name      0
city                          0
state                         0
zip_code                      0
non_us_str_code               0
country                       0
ticket_issued_date            0
hearing_date                  0
violation_code                0
violation_description         0
disposition                   0
fine_amount                   0
admin_fee                     0
state_fee                     0
late_fee                      0
discount_amount               0
clean_up_cost                 0
judgment_amount               0
grafitti_status               0
dtype: int64

In [151]:
print(X_encoded.isna().values.any())
print(X_encoded.isnull().values.any())
print(X_encoded.shape)

False
False
(250306, 26)


In [152]:
print(X_encoded.shape)
X_encoded.head()

(250306, 26)


,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,mailing_address_str_number,mailing_address_str_name,city,state,...,violation_description,disposition,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,grafitti_status
0,22056,0,137,52230,2900.0,1640,3.0,27078,682,16,...,77,6,250.0,20.0,10.0,25.0,0.0,0.0,305.0,-1
1,27586,0,166,77135,4311.0,306,2959.0,18442,1204,24,...,54,7,750.0,20.0,10.0,75.0,0.0,0.0,855.0,-1
2,22062,0,137,94185,1449.0,1021,23658.0,21873,1104,24,...,77,2,250.0,0.0,0.0,0.0,0.0,0.0,0.0,-1
3,22084,0,137,74625,1441.0,1021,5.0,28889,1104,24,...,77,0,250.0,0.0,0.0,0.0,0.0,0.0,0.0,-1
4,22093,0,137,77944,2449.0,343,7449.0,5401,1104,24,...,77,2,250.0,0.0,0.0,0.0,0.0,0.0,0.0,-1


In [153]:
Y.fillna(value=2, inplace=True)

In [154]:
print(Y.shape)
Y.head()

(250306,)


0    0.0
1    1.0
2    2.0
3    2.0
4    2.0
Name: compliance, dtype: float64

In [124]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [155]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded, Y, random_state=0)

In [156]:
%%time
clf = RandomForestClassifier().fit(X_train, y_train)

C:\Users\Admin\Anaconda3\envs\TensorFlow-GPU-Keras\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Wall time: 2.87 s


In [158]:
y_predicted = clf.predict(X_test)

In [159]:
clf.score(X_test, y_test)

0.9649232145996133

In [167]:
print((y_predicted != y_test).sum())
print((y_predicted == y_test).sum())

2195
60382


In [169]:
((y_predicted != y_test).sum() / (y_predicted == y_test).sum()) * 100

3.6351892948229603

In [170]:
from sklearn.metrics import roc_auc_score

In [180]:
y_test_auc = y_test[y_test != 2.0]
y_predicted_auc = y_predicted[y_predicted != 2.0]

In [181]:
roc_auc_score(y_test_auc, y_predicted_auc)

0.6579614188061115